In [4]:
import os
import getpass

api_key = getpass.getpass("لطفاً API Key خود را وارد کنید: ")
os.environ["OPENAI_API_KEY"] = api_key


In [5]:
import requests

def fetch_markets():
    url = "https://api.wallex.ir/hector/web/v1/markets"
    response = requests.get(url)
    response.raise_for_status()
    return response.json().get("result", {}).get("markets", [])

# تست
markets = fetch_markets()
print(f"تعداد ارزها در بازار: {len(markets)}")


تعداد ارزها در بازار: 369


In [20]:
from langchain.chat_models import init_chat_model
from langchain.memory import ConversationBufferMemory

model = init_chat_model("gpt-4o-mini", model_provider="openai")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [21]:
from langchain.agents import Tool

def search_coin(coin_name: str):
    markets = fetch_markets()
    for m in markets:
        if m.get("en_base_asset") == coin_name:
            return m
    return f"ارز {coin_name} در داده‌های API پیدا نشد."

tools = [
    Tool(
        name="SearchCoin",
        func=search_coin,
        description="برای پیدا کردن اطلاعات کامل یک ارز مشخص از API بازار"
    )
]


In [22]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    tools=tools,
    llm=model,
    agent="chat-conversational-react-description",
    memory=memory,
    verbose=True
)


In [24]:
coin_to_predict = input("لطفاً نام ارز مورد نظر را وارد کنید: ")

response = agent.run(
f"""
شما یک تحلیلگر حرفه‌ای بازار کریپتوکارنسی هستید. کاربر می‌خواهد برای ارز {coin_to_predict} پیش‌بینی قیمت ۲۴ ساعت آینده انجام شود. 

لطفاً تحلیل کاملی ارائه بده، شامل موارد زیر:
1. روند قیمت اخیر و تغییرات ۲۴ ساعت گذشته.
2. تحلیل بر اساس فیچرهای موجود مانند حجم معاملات، قیمت آخرین معامله، سقف و کف روزانه و هر فیچر دیگری که از API گرفته شده است.
3. پیش‌بینی قیمت ۲۴ ساعت آینده به صورت تقریبی.
4. دلایل و توضیحاتی که برای پیش‌بینی استفاده کردی.
5. پاسخ را به **فارسی روان و قابل فهم برای کاربر** بده.

اطلاعات ارز را از داده‌های API والکس استفاده کن و حتما تحلیل و پیش‌بینی را با هم ترکیب کن.
"""
)

print("\n=== پاسخ Agent ===")
print(response)




> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-O4VgOLLrNLXdcB1UJKNtnoWG on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}